<a href="https://colab.research.google.com/github/JADukeAIPI/AIPIProject/blob/main/LuxuryCarAPIMultipleDates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AIPI 510 PROJECT

This will allow for generating a dataframe of 'luxury' cars for several dates (currently set up to start from today and checks for 24 hour bookings for each day for next x days). The groupby at the end shows the number of vehicles available for each day, which could indirectly give us how many reservations if we assume the cars are picked up and returned in atlanta and the fleet inventory is stable.

In [37]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import requests
import datetime
from datetime import datetime, timedelta



In [38]:
today_date = datetime.today().strftime('%Y-%m-%d')

def get_date(number_days_from_today):

  end_date = datetime.today() + timedelta(days=number_days_from_today)
  end_date=end_date.strftime('%Y-%m-%d')
  return end_date



Getting Booking.com Rental Car API:


In [39]:
def get_booking_data(start_date, end_date):
	url = "https://booking-com.p.rapidapi.com/v1/car-rental/search"
	querystring = {"drop_off_longitude":"-84.419853","currency":"USD","sort_by":"recommended","drop_off_datetime":end_date + ' 12:00:00',"drop_off_latitude":"33.640411","from_country":"it","pick_up_longitude":"-84.419853","locale":"en-gb","pick_up_datetime":start_date + " 12:00:00","pick_up_latitude":"33.640411"}

	headers = {
		"X-RapidAPI-Key": "8bbb18c5b5mshcb2649ca8814f86p1d30a3jsn5a338ac42c22",
		"X-RapidAPI-Host": "booking-com.p.rapidapi.com"}

	response = requests.request("GET", url, headers=headers, params=querystring)
	results = response.json()
	df = pd.json_normalize(results, record_path =['search_results'])

	return df


In [40]:
def clean_api_df(data_df):
  df_cleaned = data_df.copy()
  
  #filter to the columns we want to keep
  columns_to_keep = ['vehicle_info.v_id', 'vehicle_info.v_name', 'vehicle_info.group', 'vehicle_info.transmission', 'route_info.dropoff.city', 'pricing_info.base_price', 'pricing_info.price', 'route_info.pickup.city']
  #filter to the types of vehicles in vehicle.group
  vehicle_groups_to_keep = ['Luxury']
  df_cleaned=df_cleaned[columns_to_keep]
  df_cleaned=df_cleaned[df_cleaned['vehicle_info.group'].isin(vehicle_groups_to_keep)]
  df_cleaned.rename(columns={'vehicle_info.v_id':'Vehicle_id', 'vehicle_info.v_name': 'Vehicle Name', 'vehicle_info.group': 'Category', 'pricing_info.base_price': 'Base Price', 'pricing_info.price': 'Price', 'vehicle_info.transmission': 'Transmission', 'route_info.dropoff.city': 'Dropoff City', 'route_info.pickup.city': 'Pickup City'}, inplace=True)
  df_cleaned.sort_values(by='Base Price', ascending=False, inplace=True)
  return df_cleaned

In [42]:
#data_trial = get_booking_data('2022-11-20', '2022-11-21')
clean_api_df(get_booking_data('2022-11-30', '2022-12-01'))


,Vehicle_id,Vehicle Name,Category,Transmission,Dropoff City,Base Price,Price,Pickup City
337,673928006,Mercedes-Benz GLC,Luxury,Automatic,,217.87,225.09,
335,723428946,BMW 5 Series,Luxury,Automatic,,200.32,206.96,
243,644881573,Cadillac XTS,Luxury,Automatic,,122.76,126.83,
225,636479888,Cadillac XTS,Luxury,Automatic,,116.24,120.09,
198,646119413,Cadillac XTS,Luxury,Automatic,,110.47,114.13,
165,644901168,Cadillac XTS,Luxury,Automatic,,105.36,108.85,
194,636492688,Cadillac XTS,Luxury,Automatic,,98.96,102.24,
184,644861493,Cadillac XTS,Luxury,Automatic,,93.77,96.88,
187,646151293,Cadillac XTS,Luxury,Automatic,,93.19,96.28,
176,658137488,Chrysler 300,Luxury,Automatic,,88.81,91.75,


In [61]:
def get_bookings_multiple_dates(days=30):
  dfs = []
  for day in range(days):
    date_tuple = (get_date(day), get_date(day+1))
    #print(date_tuple)
    df_partial = clean_api_df(get_booking_data(*date_tuple))
    df_partial['Date Pickup']=date_tuple[0]
    dfs.append(df_partial)

  df = pd.concat([dfs[i] for i in range(len(dfs))], axis=0) 

  return df



In [62]:
df_next_20_days = get_bookings_multiple_dates(20)
df_next_20_days

,Vehicle_id,Vehicle Name,Category,Transmission,Dropoff City,Base Price,Price,Pickup City,Date Pickup
240,644881573,Cadillac XTS,Luxury,Automatic,,146.40,151.25,,2022-11-16
238,636479888,Cadillac XTS,Luxury,Automatic,,139.50,144.12,,2022-11-16
225,646119413,Cadillac XTS,Luxury,Automatic,,133.53,137.96,,2022-11-16
212,644901168,Cadillac XTS,Luxury,Automatic,,128.33,132.58,,2022-11-16
217,636492688,Cadillac XTS,Luxury,Automatic,,121.58,125.61,,2022-11-16
...,...,...,...,...,...,...,...,...,...
43,735895261,Cadillac XTS,Luxury,Automatic,,70.14,72.47,,2022-12-04
50,638489136,Chrysler 300,Luxury,Automatic,,65.97,68.16,,2022-12-04
48,638446461,Chrysler 300,Luxury,Automatic,,65.97,68.16,,2022-12-04
12,631938698,Buick LaCrosse,Luxury,Automatic,,63.44,65.54,,2022-12-04


In [63]:
cars_available = df_next_20_days.groupby('Date Pickup')['Vehicle Name'].count()
cars_available

Date Pickup
2022-11-16    13
2022-11-17    15
2022-11-18    14
2022-11-19    14
2022-11-20    14
2022-11-21    26
2022-11-22    15
2022-11-23     1
2022-11-24     1
2022-11-25    14
2022-11-26    14
2022-11-27    26
2022-11-28    26
2022-11-29    26
2022-11-30    26
2022-12-01    26
2022-12-02    26
2022-12-03    26
2022-12-04    26
Name: Vehicle Name, dtype: int64